<a href="https://colab.research.google.com/github/JMMoses2190/SpideringExample/blob/main/SpideringEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import re
import itertools
from urllib.error import URLError, HTTPError, ContentTooShortError
from urllib.parse import urljoin
import pandas as pd
import time
from bs4 import BeautifulSoup
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
from gspread_dataframe import set_with_dataframe

In [2]:
def download(url, user_agent = 'wswp', num_retry = 2, charset = 'UTF-8'):
  """Downloads the HTML of the given URL 
    """
    #print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
     resp = urllib.request.urlopen(request)
     cs = resp.headers.get_content_charset()
     if not cs:
         cs = charset
     html = resp.read().decode(cs)
     #print(html)
  except(URLError, HTTPError, ContentTooShortError) as e:
             print('Download error:', e.reason)
             html = none
             if num_retry>0:
                 if hasattr(e, 'code') and 500 <=e.code<600:
                     return download(url,num_retry -1)                    
  return html 


In [3]:
def get_links(html):
  #Return a list of links from html
  #A regex to extract all links from webpage
  webpage_regex = re.compile("""<a[^>]+href=["'](.*?)["']""", re.IGNORECASE)
  #list of all links from the webpage
  return webpage_regex.findall(html) 

In [4]:
#Connect to the Google Sheet as provided by Google Colab documentation
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('JobInfo').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Job Title,JobID,JobURL,jobDesc,salaryString,postDate,facilityName,address,city,state,zip,phone,jobType
1,Irving ISD - Delivery Helper,8821,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nPrimary Pur...,Per Hour,"Tuesday, January 12, 2021 12:00 AM",Facility Services,2621 W. Airport Freeway,Irving,TX,75062,972-600-5000,Full-Time
2,Irving ISD - Energy Management Technician,8824,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nFacilites 0...,Per Hour,"Wednesday, January 13, 2021 12:00 AM",Facility Services,2621 W. Airport Freeway,Irving,TX,75062,972-600-5000,Full-Time
3,Irving ISD - HVAC Technician I\t,8827,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nPay- Facili...,Per Hour,"Thursday, January 14, 2021 12:00 AM",Facility Services,2621 W. Airport Freeway,Irving,TX,75062,972-600-5000,Full-Time
4,Irving ISD - 2021-2022 SY: 3rd Grade Bilingual...,8900,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nPRIMARY PUR...,Per Year,"Friday, March 12, 2021 12:00 AM",Good Elementary School,1200 Union Bower Rd,Irving,TX,75061,972-600-3300,Full-Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Irving ISD - 2021-2022 SY: Career & Technical ...,10881,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\n \nJOB TITL...,Per Year,"Thursday, August 19, 2021 12:00 AM",Barbara Cardwell Career Preparatory Center,2621 W. Airport Freeway,Irving,TX,75062,972-600-5000,Full-Time
173,Irving ISD - 2021-2022 SY: PreK Bilingual/ESL ...,10882,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nQUALIFICATI...,$12.85 - $16.07 / Per Hour,"Thursday, August 19, 2021 12:00 AM",Good Elementary School,1200 Union Bower Rd,Irving,TX,75061,972-600-3300,Full-Time
174,Irving ISD - Temporary Language Tester,10883,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nPRIMARY PUR...,Per Hour,"Thursday, August 19, 2021 12:00 AM",Language &amp; Parent Services,2621 W. Airport Freeway,Irving,TX,75062,972-600-5000,Full-Time
175,Irving ISD - 2021-2022 SY: MS ELAR - LB Johnso...,10884,https://irvingisd.tedk12.com/hire/ViewJob.aspx...,\r\n\r\n \r\n \n\n\n\n\n\nIRVING INDE...,Per Year,"Thursday, August 19, 2021 12:00 AM",Lady Bird Johnson Middle School,3601 W. Pioneer,Irving,TX,75061,972-600-0500,Full-Time


In [5]:

crawl_queue = ['https://irvingisd.tedk12.com/hire/Index.aspx?JobListAJAX=Paging&StartIndex=0&ListID=JobList&SearchString=']


def link_crawler(start_url, link_regex, count):
  """Go through each link on a given page
      Using the given link_regex, extract all links that match
    """
  #Setting a queue so we can check to make sure this is a new page
  seen = set(crawl_queue)
  while crawl_queue:
    url = crawl_queue.pop()
    html = download(url)
    if not html:
      continue
    for link in get_links(html):
      if re.match(link_regex, link):
        abs_link = urljoin(start_url, link)
        if abs_link not in seen:
          seen.add(abs_link)
          crawl_queue.append(abs_link)
      pagination_test = 'Last Page'
    #Some job boards are more than on page long, if this URL exists on site then there are more pages to go through
    if pagination_test in html:
      count=count+1
      start_urlA = 'https://irvingisd.tedk12.com/hire/Index.aspx?JobListAJAX=Paging&StartIndex='
      start_urlB = '&ListID=JobList&SearchString='
      start_urlC = count*25
      pagination_url = '%s%d%s' % (start_urlA, start_urlC, start_urlB)
      crawl_queue.append(pagination_url)
      link_crawler(pagination_url, link_regex, count)
    else:
       
       return crawl_queue
      

In [6]:
def get_JobsList():
  """Gets all of the jobs currently stored from prior runs
    """
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  worksheet = gc.open('JobInfo').sheet1

  # get_all_values gives a list of rows.
  rows = worksheet.get_all_values()
  

  # Convert to a DataFrame and render.

  pd.DataFrame.from_records(rows)
  print(rows)
  return rows


def get_JobInfo(url_queue):
  """Gets all relevant information from the job details page
    """
  jobHeaders = ["Job Title", "JobID","JobURL", "jobDesc","salaryString", 
                "postDate", "facilityName", "address", "city", "state",
                "zip", "phone", "jobType"]
  jobInfo = pd.DataFrame(columns = jobHeaders)
  currentList = get_JobsList()
  df = pd.DataFrame(currentList, columns= jobHeaders)


  for x in url_queue:
   url = url_queue.pop()
   html = download(url)

   #Get jobID from the URL
   jobID = re.findall('JobID=(\d+)', url)
   jobID=str(jobID)[2:-2]

    
   #Find the jobTitle
   jobTitle = re.findall('<meta property="og:title" content="(.*?)"/>', html)
   jobTitle=str(jobTitle)[2:-2]

   """Job Description is more complicated
        It is downloaded from a seperate page
        We need to download the page's HTML first 
    """
   jobDescURLB = re.findall('<iframe src="(.*?)"', html)
   jobDescURLB=str(jobDescURLB)[2:-2]
   jobDescURLA = "https://irvingisd.tedk12.com/hire/"
   jobDescURL = '%s%s' % (jobDescURLA, jobDescURLB)
   jobDesc = download(jobDescURL)
   
   """Using BeaultifulSoup
        Downloading the HTML for the extra jobDesc page
        Using .get_text() to convert HTML to text
    """
   jobDesc = BeautifulSoup(jobDesc)
   jobDesc = jobDesc.get_text()
   jobDescFinal = jobDesc.split('});')

   #Find salaryString
   salaryString = re.findall('<span id="lblSalary">(.*?)</span', html)
   salaryString=str(salaryString)[2:-2]

   #Find postDate
   postDate = re.findall('<span id="lblApplicationStartDate">(.*?)</span', html)
   postDate=str(postDate)[2:-2]

   #Find jobType
   jobType = re.findall('<span id="lblShiftType">(.*?)</span', html)
   jobType=str(jobType)[2:-2]

   """Site only gives the FacilityName
        Checking each possible location and assigning proper geo info
    """
   facilityName = re.findall('<span id="lblLocationName">(.*?)</span', html)
   facilityName=str(facilityName)[2:-2]
   if facilityName == 'Austin Middle School':
     address = '825 E. Union Bower'
     city = 'Irving'
     state = 'TX'
     zip = '75061'
     phone = '972-600-3100'
   elif facilityName == 'Barton Elementary School':
     address = '2931 Conflans'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-4100'
   elif facilityName == 'Bowie Middle School':
     address = '600 E. Sixth'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-3000'
   elif facilityName == 'Britain Elementary School':
     address = '631 Edmondson'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-3800'
   elif facilityName == 'Clifton Early Childhood School':
     address = '3950 Pleasant Run'
     city = 'Irving'
     state = 'TX' 
     zip = '75038'
     phone = '972-600-4200'
   elif facilityName == 'Crockett Middle School':
     address = '2431 Hancock'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-4700'
   elif facilityName == 'Davis Elementary School':
     address = '310 Davis Drive'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-4900'
   elif facilityName == 'de Zavala Middle School':
     address = '707 W. Pioneer'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-6000'
   elif facilityName == 'Elliott Elementary School':
     address = '1900 S. Story'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-4300'
   elif facilityName == 'Gilbert Elementary School':
     address = '1501 E. Pioneer'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-0400'
   elif facilityName == 'Good Elementary School':
     address = '1200 Union Bower Rd'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-3300'
   elif facilityName == 'Hanes Elementary School':
     address = '2730 Cheyenne'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-3600'
   elif facilityName == 'Houston Middle School':
     address = '3033 W. Country Club'
     city = 'Irving'
     state = 'TX' 
     zip = '75068'
     phone = '972-600-7500'
   elif facilityName == 'Irving High School':
     address = '900 N. O\'Connor'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-6300'
   elif facilityName == 'John Haley Elementary School':
     address = '1100 Schulze'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-6600'
   elif facilityName == 'Johnston Elementary School':
     address = '2801 Rutgers'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-7700'
   elif facilityName == 'Kinkeade Early Childhood School':
     address = '2333 Cameron Place'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-6500'
   elif facilityName == 'Lady Bird Johnson Middle School':
     address = '3601 W. Pioneer'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-0500'
   elif facilityName == 'Lamar Middle School':
     address = '219 Crandall'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-4400'
   elif facilityName == 'Lee Elementary School':
     address = '1600 Carlisle'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-7800'
   elif facilityName == 'Lively Elementary School':
     address = '1800 Plymouth Drive West'
     city = 'Irving'
     state = 'TX' 
     zip = '75061'
     phone = '972-600-6700'
   elif facilityName == 'MacArthur High School':
     address = '3700 N. MacArthur'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-7200'
   elif facilityName == 'Nimitz High School':
     address = '100 West Oakdale'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-5700'
   elif facilityName == 'Singley Academy':
     address = '4601 N. MacArthur'
     city = 'Irving'
     state = 'TX' 
     zip = '75038'
     phone = '972-600-5300'
   elif facilityName == 'Stipes Elementary':
     address = '3100 Cross Timbers'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-4500'
   elif facilityName == 'Thomas Haley Elementary School':
     address = '3601 Cheyenne'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-7000'
   elif facilityName == 'Townley Elementary School':
     address = '1030 Vilbig'
     city = 'Irving'
     state = 'TX' 
     zip = '75060'
     phone = '972-600-6800'
   elif facilityName == 'Travis Middle School':
     address = '1600 Finley'
     city = 'Irving'
     state = 'TX' 
     zip = '75062'
     phone = '972-600-0100'
   #If nothing is true, just assign geo as district location
   else:
     address = '2621 W. Airport Freeway'
     city = 'Irving'
     state = 'TX'
     zip = '75062'
     phone = '972-600-5000'
      

   
   jobInfo.loc[len(jobInfo.index)] = [jobTitle, jobID, url, jobDescFinal[5], 
                                      salaryString, postDate, facilityName, 
                                      address, city, state, zip, phone, jobType] 
   url_queue.insert(0, url)

  return jobInfo

  


In [7]:
def update_data(jobInfo):
  """Updated data in the Google Sheet
    """
  gc = gspread.authorize(GC.get_application_default())
# create, and save df
  title = 'JobInfo'
  sheet = gc.open(title).sheet1
  set_with_dataframe(sheet, jobInfo) 
# include_index=False, include_column_header=True, resize=False

In [8]:
pagination = 1
url = 'https://irvingisd.tedk12.com/hire/Index.aspx?JobListAJAX=Paging&StartIndex=0&ListID=JobList&SearchString='
url_queue = []
link_regex = '.*?JobID=\d+$'
url_queue = link_crawler(url, link_regex, pagination)
print(len(url_queue))
jobInfo = get_JobInfo(url_queue)
if not jobInfo.empty:
  update_data(jobInfo)



176
[['Job Title', 'JobID', 'JobURL', 'jobDesc', 'salaryString', 'postDate', 'facilityName', 'address', 'city', 'state', 'zip', 'phone', 'jobType'], ['Irving ISD - Delivery Helper', '8821', 'https://irvingisd.tedk12.com/hire/ViewJob.aspx?JobID=8821', '\r\n\r\n       \r\n    \n\n\n\n\n\nPrimary Purpose:Assist in receiving, storing, issuing and delivering of equipment, tools, and supplies and maintaining inventory control.\nQualifications:Education/CertificationHigh School Diploma or GEDValid Class C Texas driver’s license and proof of insurability with District insurance carrier\n\xa0\nSpecial Knowledge/SkillsAbility to perform multiple tasks simultaneouslyAbility to understand and follow detailed written or verbal instructionsAbility to perform basic math accurately and quicklyStrong organizational, communication (verbal and written), and interpersonal skills\n\xa0\nExperience\nSix months experience warehouse and/or delivery preferred\n\xa0\nMajor Responsibilities and Duties:Receiving 